In [1]:
%load_ext watermark
%watermark -v -m -p pandas,numpy,scipy,faker -g

CPython 3.5.3
IPython 5.1.0

pandas 0.19.2
numpy 1.11.3
scipy 0.19.0
faker n

compiler   : GCC 4.4.7 20120313 (Red Hat 4.4.7-1)
system     : Linux
release    : 4.4.0-79-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
Git hash   : e4d423ecd94684bb81768e5b7d1b692cd6a43a7b


In [3]:
shows = {'title':['Velocity','HD','Punky Heels','Jack Wild','Duendes','Cant Buy Time'],
        'genre':['action','drama','comedy','reality','fantasy','sci-fi'],
        'description':['an action packed thriller about a pregnant woman stuck on a subway car that cannot slow down',
                      '35 year old Mike, struggles to reconnect with his birth family after being diagnosed with a genetic disorder',
                      'a romantic comedy about a young woman trying to find love in the city',
                      'a recent college grad trying to find himself learns the meaning of life while stuck in an alaskan blizzard',
                      'an unknown creature terrorizes the town of Salamanca, will they survive?',
                      'a group of friends gets stuck in a suburban shopping mall as the manifold of time and space collapses around them'
                      ],
        'show_id':[1,2,3,4,5,6]
        }

In [7]:
import pandas as pd

show_df = pd.DataFrame(shows)

In [8]:
show_df

,description,genre,show_id,title
0,an action packed thriller about a pregnant wom...,action,1,Velocity
1,"35 year old Mike, struggles to reconnect with ...",drama,2,HD
2,a romantic comedy about a young woman trying t...,comedy,3,Punky Heels
3,a recent college grad trying to find himself l...,reality,4,Jack Wild
4,an unknown creature terrorizes the town of Sal...,fantasy,5,Duendes
5,a group of friends gets stuck in a suburban sh...,sci-fi,6,Cant Buy Time


In [22]:
from faker import Faker
fake = Faker()
people = {'name': [],
          'sex': [],
          'bday': [],
          'address': [],
          'p_id':[]
         }

for i in range(10):
    profile = fake.simple_profile(sex=None) 
    people['name'].append(profile['name'])
    people['sex'].append(profile['sex'])
    people['bday'].append(profile['birthdate'])
    people['address'].append(profile['address'])
    people['p_id'].append(i)
    
people_df = pd.DataFrame(people)

In [65]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [140]:
np.random.uniform(0,10)

3.592140826371412

In [143]:
def generate_alpha_dict(x, y):
    output = {}
    if isinstance(y, list):
        low_y = y[0]
        high_y = y[1]
    else:
        low_y = 1
        high_y = y
        
    for i in range(x+1):
        output[i] = np.random.uniform(low_y,high_y)
    
    return output

In [156]:
import numpy as np
import datetime

age_tv_watching_alphas = generate_alpha_dict(10,10)

def create_use_prob(customer_df, alpha=5, beta=5, date_column='bday'):
    assert isinstance(customer_df, pd.DataFrame)
    
    def find_age_group(row, alpha):
        assert isinstance(alpha, dict)
        
        def calculate_age(born):
            assert isinstance(born, datetime.datetime)
        
            today = datetime.date.today()
            return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
    
        born = datetime.datetime.strptime(row[1][date_column],'%Y-%m-%d')
        age = int(calculate_age(born) / 10)
        while True:
            if age in alpha:
                return alpha[age]
            else:
                age -= 1
            if age < 0:
                return 'alpha_dict broken!'
    
    watch_prob = []
    love_tv = []
    for i in customer_df.iterrows():
        watch_propensity = 0
        while watch_propensity <= 0:
            watch_propensity = find_age_group(i, alpha)
            tv_propensity = np.random.normal()*2.0
            watch_propensity += tv_propensity 
        
        love_tv.append(tv_propensity)
        watch_prob.append(np.random.beta(watch_propensity, beta))
    
    customer_df['affinity_4_tv'] = love_tv
    customer_df['watch_prob'] = watch_prob
    return customer_df

In [157]:
people_df = create_use_prob(people_df, alpha=age_tv_watching_alphas)

In [158]:
people_df

,address,bday,name,p_id,sex,watch_prob,affinity_4_tv
0,"PSC 9059, Box 3964\nAPO AP 58127",1998-11-21,Patrick Reed,0,M,0.638578,-2.053810
1,"37793 Reed Passage Suite 833\nDawnton, PA 78777",2013-04-06,Jimmy Moran,1,M,0.195426,-2.936765
2,"8880 Kim Drives\nLake Cody, CO 62943-2779",1975-09-15,Michael Williams,2,M,0.292500,-2.443869
3,"83356 Alexander Village Apt. 787\nAnthonyfurt,...",2014-05-10,Andrea Ayala,3,F,0.599566,1.376630
4,Unit 0163 Box 6168\nDPO AE 50838,1974-07-27,Wendy Lloyd,4,F,0.654158,1.534502
5,Unit 8156 Box 1258\nDPO AE 95424,1997-08-15,Christopher Gonzalez,5,M,0.644682,-0.834553
6,"263 Mahoney Squares\nEast Walterburgh, WV 0024...",1983-07-18,Julie Owens,6,F,0.567360,0.986996
7,"5854 Cobb Loop Apt. 896\nNorth Casey, KS 40699",1996-01-23,Lindsay Johnson,7,F,0.009072,-3.785448
8,USS Shepherd\nFPO AA 91039-0685,1988-08-16,Autumn Young,8,F,0.310576,-0.118203
9,"894 Reyes Flat Suite 144\nSarahbury, MT 59905-...",1979-07-24,Emily Moreno,9,F,0.493374,-2.437766


In [120]:
np.where(np.random.multinomial(1, [1/6.]*6, size=1)[0])[0][0]

0

create another function that maps alpha values to individuals, but this time one alpha value for each show. This function should use age and gender and maybe another hidden variable. I can then give the alpha values to a dirichlet which will give probabilities of watched each show, which is given to the multinomal function, which chooses which show the individual watched. 

after I know what shows people watched, I can just count the number of times a person watched a show. The more times a person watches a show, the more likely they are to sign up. This will be some final binomial draw where the probability is drawn from a beta whose alpha is determined by the number of shows watched.

after all this i have to add in bad data somehow. then just think about how to make this generalize better!

In [128]:
np.random.dirichlet((10, 5.3, 3, 4), 1)[0]

array([ 0.26648385,  0.40599903,  0.10231488,  0.22520225])

In [164]:
i[1]['sex']

'F'

In [198]:
female_alphas = generate_alpha_dict(10,10)
male_alphas = generate_alpha_dict(10,10)

def find_show_probability(row, alpha_dict):
    male_dict = alpha_dict[0]
    female_dict = alpha_dict[1]
    gender = row[1]['sex']
    if gender == 'M':
        alpha = male_dict
    elif gender == 'F':
        alpha = female_dict
    else:
        return 'i am outdated and only do binary gender'
    
    born = datetime.datetime.strptime(row[1]['bday'],'%Y-%m-%d')
    age = int(calculate_age(born) / 10)
    
    while True:
        if age in alpha:
            return [alpha[age], 5]
        else:
            age -= 1
        if age < 0:
            return 'alpha_dict broken!'

In [199]:
def per_row_draws(customer_df, draw_func, parameter_func, alpha):
    assert isinstance(customer_df, pd.DataFrame)
    
    output = []
    for i in customer_df.iterrows():
        draw_inputs = parameter_func(i, alpha)
        output.append(draw_func(*draw_inputs))
    
    return output

In [200]:
female_alphas = generate_alpha_dict(10,10)
male_alphas = generate_alpha_dict(10,10)

alpha = [male_alphas, female_alphas]

per_row_draws(people_df, np.random.normal, find_show_probability, alpha)

[7.240558165011662,
 17.498622220849953,
 5.601372470734555,
 12.358945627677803,
 14.260092128748772,
 2.6559956096621278,
 12.509482712510042,
 6.030096790495744,
 10.837648101103587,
 4.258367446116318]

In [ ]:
def create_use_df(customer_df, product_df, size=1000, person_id = 'p_id'):
    behavior_df = pd.DataFrame()
    while behavior_df.shape[0] < size:
        person = np.random.choice(people_df[person_id])
        if np.random.binomial(1, customer_df[customer_df[person_id] == person].watch_prob.values[0]):
            show_watched = np.where(np.random.multinomial(1, [1/6.]*6, size=1)[0])[0][0]